# DNN

In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error

## 데이터 가져오기

In [47]:
train_set = pd.read_csv('train_set.csv')
train_set = train_set[train_set['ECLO'] <= 25]
test_set = pd.read_csv('test_set.csv')

## train test valid 분리

In [49]:
train_x, test_x, train_y, test_y = train_test_split(train_set[['요일', '기상상태', '도로형태', '노면상태', '사고유형']], train_set['ECLO'], test_size=0.2)

In [50]:
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2)

## tensorflow model 정의

In [51]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(5)),
    tf.keras.layers.Dense(5, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(50, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Dense(10, activation='relu', kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dense(1)]
)

In [52]:
checkpoint_filepath = "logs/DNN/"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
)

In [53]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.MeanSquaredLogarithmicError(), metrics=['mae'])
model.fit(train_x, train_y, epochs=100, validation_data=(valid_x, valid_y),
          callbacks=[
                        checkpoint_callback,
                        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3),
                    ])

Epoch 1/100
791/791 [==============================] - 10s 10ms/step - loss: 0.2312 - mae: 2.1462 - val_loss: 0.1967 - val_mae: 2.0126
Epoch 2/100
791/791 [==============================] - 8s 10ms/step - loss: 0.2008 - mae: 2.0500 - val_loss: 0.1966 - val_mae: 2.0322
Epoch 3/100
791/791 [==============================] - 8s 10ms/step - loss: 0.2007 - mae: 2.0509 - val_loss: 0.1985 - val_mae: 1.9977
Epoch 4/100
791/791 [==============================] - 8s 11ms/step - loss: 0.2009 - mae: 2.0516 - val_loss: 0.1965 - val_mae: 2.0209
Epoch 5/100
791/791 [==============================] - 8s 10ms/step - loss: 0.2011 - mae: 2.0517 - val_loss: 0.1965 - val_mae: 2.0237
Epoch 6/100
791/791 [==============================] - 8s 10ms/step - loss: 0.2008 - mae: 2.0505 - val_loss: 0.1968 - val_mae: 2.0354
Epoch 7/100
791/791 [==============================] - 8s 10ms/step - loss: 0.2010 - mae: 2.0502 - val_loss: 0.1970 - val_mae: 2.0402
Epoch 8/100
791/791 [==============================] - 8s 10m

In [54]:
np.sqrt(mean_squared_log_error(test_y, model.predict(test_x)))

248/248 [==============================] - 1s 3ms/step


0.4513449381670079

## 결과 저장